# LangChain Expression Language (LCEL) Tutorial

In [1]:
from typing import Any, Dict, List

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_ollama import ChatOllama
from rich import print

In [2]:
# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOllama(model="llama3.2",temperature=0.3)

## SECTION 1: Basic LCEL Chain

In [4]:
print("-" * 50,'\n',"DEMO 1: Basic LCEL Chain",'\n',"-" * 50)

# Create a simple prompt template
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")

# Create a basic chain using the | operator (pipe)
basic_chain = prompt | llm | StrOutputParser()


print("Prompt: ", prompt.format_prompt(topic="cats").to_messages())
print(basic_chain)
print("Chain: ", basic_chain.invoke({"topic": "cats"}))

-------------------------------------------------- 
 DEMO 1: Basic LCEL Chain 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='Tell me a short joke about cats.', additional_kwargs={}, response_metadata={})]

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Tell me a short joke about {topic}.'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[ChatOllama(model='llama3.2', temperature=0.3)],
    last=StrOutputParser()
)

Chain:  Why did the cat join a band?

Because it wanted to be the purr-cussionist!

In [5]:
# Run the chain
result = basic_chain.invoke({"topic": "programming"})
print(result)
print("\n")

Why do programmers prefer dark mode?

Because light attracts bugs.

## SECTION 2: Parallel Processing with LCEL

In [6]:
print("-" * 50,'\n',"DEMO 2: Parallel Processing with LCEL",'\n',"-" * 50)

# Create two different prompts
pros_prompt = ChatPromptTemplate.from_template(
    "List 3 pros of {technology} technology."
)
cons_prompt = ChatPromptTemplate.from_template(
    "List 3 cons of {technology} technology."
)

# Create chains for each prompt
pros_chain = pros_prompt | llm | StrOutputParser()
cons_chain = cons_prompt | llm | StrOutputParser()

# Create a parallel chain
parallel_chain = RunnableParallel(pros=pros_chain, cons=cons_chain)

print("Prompt: ", pros_prompt.format_prompt(technology="AI").to_messages())
print("Chain: ", parallel_chain)

-------------------------------------------------- 
 DEMO 2: Parallel Processing with LCEL 
 --------------------------------------------------

Prompt: 
[HumanMessage(content='List 3 pros of AI technology.', additional_kwargs={}, response_metadata={})]

Chain: 
RunnableParallel(
    steps__={
        'pros': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['technology'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['technology'],
                            input_types={},
                            partial_variables={},
                            template='List 3 pros of {technology} technology.'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[ChatOllama(model='llama3.2', temperature=0.3)],
            last=StrOutputParser()
        ),
        'cons': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['technology'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['technology'],
                            input_types={},
                            partial_variables={},
                            template='List 3 cons of {technology} technology.'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[ChatOllama(model='llama3.2', temperature=0.3)],
            last=StrOutputParser()
        )
    }
)

In [7]:

# Run the parallel chain
parallel_result = parallel_chain.invoke({"technology": "blockchain"})
print("PROS:")
print(parallel_result["pros"])
print("\nCONS:")
print(parallel_result["cons"])
print("\n")

PROS:

Here are three pros of blockchain technology:

1. **Security and Transparency**: Blockchain technology uses a decentralized ledger system that is resistant to 
tampering, hacking, and manipulation. All transactions on the blockchain are recorded in a public ledger, making it
transparent and secure.

2. **Immutable and Tamper-Proof**: The blockchain's decentralized nature and use of cryptography make it virtually 
impossible to alter or manipulate transactions once they have been recorded. This ensures that data is accurate, 
reliable, and tamper-proof.

3. **Decentralized and Autonomous**: Blockchain technology allows for a decentralized network of nodes to validate 
and record transactions, making it an autonomous system that operates independently of central authorities. This 
reduces the need for intermediaries and can lead to faster transaction processing times and lower costs.

CONS:

Here are three cons of blockchain technology:

1. **Scalability Issues**: Blockchain technology is still in its early stages, and one of the major limitations is 
its scalability. As the number of transactions increases, the blockchain can become slow and congested, leading to 
high transaction fees and long processing times.

2. **Energy Consumption**: The process of mining cryptocurrency, which is a key component of many blockchain 
networks, requires significant amounts of energy. This has raised concerns about the environmental impact of 
blockchain technology, particularly for large-scale applications.

3. **Regulatory Uncertainty**: Blockchain technology operates in a gray area between traditional financial 
regulations and new, emerging laws. This uncertainty can make it difficult for businesses to navigate regulatory 
requirements and can create risks for companies that use blockchain technology.

## SECTION 3: Advanced Chain with Data Transformation

In [8]:
print("-" * 50,'\n',"DEMO 3: Advanced Chain with Data Transformation",'\n',"-" * 50)

# Define a function to process input
def preprocess_input(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Process the input data before sending to the LLM."""
    # Add a Word count, capitalize the query, etc.
    processed_data = input_data.copy()
    processed_data["query"] = input_data["query"].upper()
    processed_data["word_count"] = len(input_data["query"].split())
    return processed_data


# Define a function to process output
def postprocess_output(output: str) -> Dict[str, Any]:
    """Process the output from the LLM."""
    lines = output.strip().split("\n")
    return {
        "summary": lines[0] if lines else "",
        "details": lines[1:],
        "response_length": len(output),
    }


# Create an advanced prompt
advanced_prompt = ChatPromptTemplate.from_template(
    """Answer the following query: {query}

    The query has {word_count} words.
    Provide a detailed explanation.
    """
)

# Create an advanced chain with preprocessing and postprocessing
advanced_chain = (
    RunnableLambda(preprocess_input)
    | advanced_prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(postprocess_output)
)

# Preprocess the input data
preprocessed_data = preprocess_input({"query": "Explain quantum computing"})

print("Prompt: ", advanced_prompt.format_prompt(**preprocessed_data).to_messages())
print("Chain: ", advanced_chain)

-------------------------------------------------- 
 DEMO 3: Advanced Chain with Data Transformation 
 --------------------------------------------------

Prompt: 
[
    HumanMessage(
        content='Answer the following query: EXPLAIN QUANTUM COMPUTING\n\n    The query has 3 words.\n    Provide a
detailed explanation.\n    ',
        additional_kwargs={},
        response_metadata={}
    )
]

Chain: 
RunnableSequence(
    first=RunnableLambda(preprocess_input),
    middle=[
        ChatPromptTemplate(
            input_variables=['query', 'word_count'],
            input_types={},
            partial_variables={},
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['query', 'word_count'],
                        input_types={},
                        partial_variables={},
                        template='Answer the following query: {query}\n\n    The query has {word_count} words.\n   
Provide a detailed explanation.\n    '
                    ),
                    additional_kwargs={}
                )
            ]
        ),
        ChatOllama(model='llama3.2', temperature=0.3),
        StrOutputParser()
    ],
    last=RunnableLambda(postprocess_output)
)

In [9]:
# Run the advanced chain
advanced_result = advanced_chain.invoke(
    {"query": "how does LCEL improve LLM applications"}
)
print(f"Summary: {advanced_result['summary']}")
print("Details:")
for detail in advanced_result["details"]:
    if detail.strip():
        print(f"- {detail.strip()}")
print(f"Response Length: {advanced_result['response_length']} characters")
print("\n")

Summary: LCE (Loss Constrained Embedding) is a technique used to improve Large Language Model (LLM) applications. 
Here's how it works:

Details:

- **What is Loss Constrained Embedding?**

- Loss Constrained Embedding is a method for training LLMs by constraining the loss function to a specific range or
distribution. The goal of this technique is to prevent the model from overfitting to the training data and to 
improve its generalization performance.

- **How does LCE improve LLM applications?**

...

- * **Dropout**: adds a penalty term to the activations of the model

- The choice of penalty term depends on the specific application and the desired behavior of the model.

- **Conclusion**

- In conclusion, Loss Constrained Embedding is a technique that improves Large Language Model applications by 
constraining the loss function to a specific range or distribution. This leads to improved generalization, reduced 
overfitting, increased robustness, improved performance on out-of-vocabulary words, and faster convergence during 
training.

Response Length: 2818 characters

## SECTION 4: RAG Pattern with LCEL

In [10]:
print("-" * 50,'\n',"DEMO 4: RAG Pattern with LCEL (Simulated)",'\n',"-" * 50)


# Simulate a retriever function
def simulate_retrieval(query: str) -> List[str]:
    """Simulate document retrieval based on query."""
    # In a real application, this would query a vector database
    documents = [
        "LCEL allows for declarative chain construction in LangChain.",
        "LangChain Expression Language simplifies building complex LLM applications.",
        "LCEL supports streaming, async operations, and parallel execution.",
    ]
    return documents


# Create a RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the following context:

    Context:
    {context}

    Question: {question}
    """
)


# Create a RAG chain
def format_docs(docs):
    return "\n".join(docs)


rag_chain = (
    RunnableParallel(
        {
            "context": RunnableLambda(simulate_retrieval) | RunnableLambda(format_docs),
            "question": RunnablePassthrough(),
        }
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("Prompt: ", rag_prompt.format_prompt(context="context", question="question").to_messages())
print("Chain: ", rag_chain)

-------------------------------------------------- 
 DEMO 4: RAG Pattern with LCEL (Simulated) 
 --------------------------------------------------

Prompt: 
[
    HumanMessage(
        content='Answer the question based on the following context:\n\n    Context:\n    context\n\n    Question: 
question\n    ',
        additional_kwargs={},
        response_metadata={}
    )
]

Chain: 
RunnableSequence(
    first=RunnableParallel(
        steps__={
            'context': RunnableSequence(
                first=RunnableLambda(simulate_retrieval),
                middle=[],
                last=RunnableLambda(format_docs)
            ),
            'question': RunnablePassthrough()
        }
    ),
    middle=[
        ChatPromptTemplate(
            input_variables=['context', 'question'],
            input_types={},
            partial_variables={},
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['context', 'question'],
                        input_types={},
                        partial_variables={},
                        template='Answer the question based on the following context:\n\n    Context:\n    
{context}\n\n    Question: {question}\n    '
                    ),
                    additional_kwargs={}
                )
            ]
        ),
        ChatOllama(model='llama3.2', temperature=0.3)
    ],
    last=StrOutputParser()
)

In [11]:
# Run the RAG chain
rag_result = rag_chain.invoke("What is LCEL and why is it useful?")
print(rag_result)

Based on the given context, LCEL stands for LangChain Expression Language. It is a tool that allows for declarative
chain construction in LangChain, which simplifies building complex Large Language Model (LLM) applications.

LCEL is useful because it supports streaming, async operations, and parallel execution, making it easier to build 
and manage complex applications that require these capabilities.